# Guide for the compilation of the INCA Core and the implementation of a simple hydrological model

This document is meant to be a hands-on guide for the compilation of the INCA Core which is at its essence a model-building tool, written in C++, that allows for rapid model prototyping at the cost of computational performance. It is meant to be deployed as a _library_ against which your program should _link_ in order to create models.

Before going through this page, the reader is encouraged to browse the existing technical manual and the _*\.pptx_ presentations describing the Core, its philosophy and implementation. These can be found on [github repository](https://github.com/biogeochemistry/INCA/tree/master/documentation) where the INCA source is stored. Please see the [preliminary requirements](#prel_req) section of this document for an explanation on how to get access to the repository.

In short, the Core is a **model-building tool** resulting from the necessity of providing a structured, clear, and easily maintainable code base with which to build  and further advance the INCA family of models. This necessity arose from 20+ years of near-organic growth of the INCA models and the limitations this imposed on maintainability.

It is written in C++ and the programming style is object oriented. One of the main arguments in favor of object-oriented programming is its ability to provide _encapsulation_, i.e. divide the code into layers of complexity. This requires careful planning for the structure of the code (and as such complicates structural changes), but allows rapid development, which is a definite benefit for model prototyping. Encapsulating data accesses and more importantly, operations on data allows to layering the degree of understanding of the C++ code required to interact with the core. 

At its outermost, or simplest, level, the Core allows for the creation of models by specifying model parameters, inputs, outputs and equations (and their linkages) and provides a variety of solvers (which can be specified and/or added). It automatically creates all looping traditionally required in time-dependent models.

In order to interact with the Core at this level it is not necessary to be fluent with C++ but familiarity with it is an advantage. This document deals with usage of the Core at this level and aims to:

* show how get the source code on your local machine.
* provide a guide on how to compile the Core.
* show how to create and run simple model using the Core.


<h1 id="nocountDemo" class="nocount">Python settings and imports</h1>

You can safely ignore this section. It is only meant to group the **imports** for the _Python_ sections of this document. 

* Importing modules required to run this notebook
* Defining functions used in this notebook
* Setting plotting options

In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib as plt

<a id="prel_req"></a>

# Preliminary requirements

## Getting the source code

- Get a [github](https://github.com/) account.
- Get access to the repository, if you don't already have it. Access is granted by either [JLG](http://www.niva.no/en/se-ansatt?navn=Jose-Luis%20Guerrero%20Calidonio) or [RMC](http://www.niva.no/en/se-ansatt?navn=Raoul-Marie%20Couture).
- Clone the repository https://github.com/biogeochemistry/INCA.git to your computer. There are many ways of achieving this and you might already have your preferences. If you don't, you could:
    - in Windows: use the [github desktop](https://desktop.github.com/) and [follow the instructions therein](https://help.github.com/desktop/guides/contributing/cloning-a-repository-from-github-to-github-desktop/).
    - in _\*nix_: a shell should suffice. Allergies to command line interfaces can be cured with, amongst many options, something like [git cola](http://git-cola.github.io/). 
- Be it through shell or GUI, it boils down to feeding github the following command together with your credentials:
<div class=warn>
**shell: ** git clone https://<span></span>github.com/biogeochemistry/INCA.git
<div/>

You should now have the source code, as well as the documentation in your local machine. 

The Core depends on two external libraries in order to run. Binaries are provided but they can also be compiled from source. This is explained in more detail in the [dependencies](#dependencies) section.

<div class=warn>
**shell: ** git clone https://<span></span>github.com/biogeochemistry/INCA.git
<div/>

<a id="dependencies"></a>
## Dependencies

### Boost

The [Boost C++ libraries](http://www.boost.org/) are a highly-regarded, peer-reviewed  collection of libraries, some of them being of such high-quality and usefulness that they end up becoming part of the C++ standard.

Most Boost libraries are header-only, i.e. need only be placed in a _PATH_ accessible to your compiler (there are several ways of achieving this) but others need to be _linked_ and as such require prior compilation themselves. 

The Core requires the [boost::datetime](http://www.boost.org/doc/libs/1_63_0/doc/html/date_time.html) library which therefore needs to be obtained or compiled from source beforehand.

For windows, [binaries](https://sourceforge.net/projects/boost/files/boost-binaries/1.63.0/) can be obtained requiring no compilation.

Instructions to compile the Boost libraries can be found [here](http://www.boost.org/doc/libs/1_63_0/more/getting_started/windows.html) for Windows and [here](http://www.boost.org/doc/libs/1_63_0/more/getting_started/unix-variants.html) for _\*nix_.

A general piece of advice would be to use the same compiler throughout. This goes a long way to prevent linking problems. 

The Boost libraries are meant to be backwards compatible but, to put it bluntly, shit happens. The Core has been tested with versions 1_59_0 through 1_62_0.

### TinyXML2

[TinyXML2](http://www.grinninglizard.com/tinyxml2/) is a lightweight XML parser that is used within the Core to read input files and/or generate output, although the latter is not recommended as it is not an efficient way of generating output.

It is available for download from [github](https://github.com/leethomason/tinyxml2).

### A quick discussion on linking 

If your code is to make use of a library, it has to _link_ (point) to it. This linkage can be either _static_ or _dynamic_. With static linking the library is included within every copy (executable) of your compiled program whereas with dynamic linking the library is placed within a folder specified in the PATH _environment variable_.

In Windows you can [modify the PATH following this procedure.](http://superuser.com/questions/949560/how-do-i-set-system-environment-variables-in-windows-10)

In _\*nix_ you can:
<section>
   <div class="warn">
   **shell: ** echo $PATH
   <div/>
</section>

to see which folders are already included (so you place your library in one of them) or modify it to add a path of your convenience:

<section>
   <div class="warn">
   **shell: ** PATH=/your/path:$PATH
   <div/>
</section>

Static linking makes distribution and installation of the executable simpler, since all is contained with a single package. Dynamic linking makes compilation simpler, as a change in a library would only require a re-compilation of that library but not of the entire source. 

The instructions given here are for static linking.

## Compilers

A compiler converts code into machine language that your computer can understand. When it comes to C++ compilers, most follow the C++ standard, which is a set of instructions on how the code is to be converted to machine language. The standard is constantly evolving and might in some cases be subject to _interpretation_. 

Furthermore, different compilers follow different _philosophies_ on how code should or can be transformed into _assembly_ (the language your processor understands). Some have their own ideosyncrasies or even extensions of the standard.

As such, combining libraries born of different compilers, or even compiler _versions_ is often a losing proposition or, if the stars align, a serious challenge to your sanity. 

The _standard_ that is followed by the code in the Core is called C++11. Depending on your compiler, this might be the default, needs to be activated with a _flag_ or might be altogether unavailable or only partially implemented.

For the purpose of this document the compilers that were tested are:
- in Windows: mingw
- in _\*nix_: gcc-5.4.0

The technical manual for the Core states that the supported compilers are

In [2]:
# Path to custom CSS style for this document

from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [3]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

<IPython.core.display.Javascript object>